# Importing Essential Libraries

In [ ]:
import os, sys
import pandas as pd
import geopandas as gpd
import shapely
import numpy as np
import bokeh.models as bkhm
import bokeh.colors as bokeh_colors

sys.path.append(os.path.abspath('../../src')) # Ideally ST_Visions will be installed as a module, therefore this is more of a development setting for quick testing.

from st_visions.visualization.st_visualizer import st_visualizer
import st_visions.express as viz_express
import st_visions.visualization.geom_helper as viz_helper
import st_visions.visualization.providers as viz_providers
import st_visions.visualization.callbacks as viz_callbacks

os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "*" # DEV ENVIROMENT BANDAID FOR VSCODE VISUALIZATIONS, will be edited

pd.set_option('display.expand_frame_repr', False)

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Defining Global Variables

  * ### Loading Dataset

In [ ]:
gdf = pd.read_csv(r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv')
gdf = viz_helper.create_geometry(gdf, crs=4326)

  * ### Creating Choropleth (Grid) Geometry

  We get the borderbox of an existing GeoDataFrame, then use shapely to create a Polygon covering the box (a square essentially)

In [ ]:
# get bounding box as (minx, miny, maxx, maxy)
bbox = gdf.total_bounds
west, south, east, north = bbox

polygon_corners = [
    (west, north),
    (east, north),
    (east, south),
    (west, south)
]

bbox_polygon = shapely.geometry.Polygon(polygon_corners)
cut_result = viz_helper.quadrat_cut_geometry(bbox_polygon, 0.7)

spatial_coverage_cut = gpd.GeoDataFrame(
    geometry=list(cut_result.geoms) if hasattr(cut_result, 'geoms') else [cut_result],
    crs=4326
)


  * ### Classifying Area Proximity (i.e., Populate the Choropleth Map)

In [ ]:
#Classify points into spatial areas
classified_gdf = viz_helper.classify_area_proximity(gdf.copy(), spatial_coverage_cut, compensate=True, verbose=True)

# Get how many points are in each area
cnt = classified_gdf['area_id'].value_counts()

# Assign counts to spatial_coverage_cut GeoDataFram. Note: spatial_coverage_cut index should match the area_id values
spatial_coverage_cut = spatial_coverage_cut.copy() 
spatial_coverage_cut['count'] = 0 
spatial_coverage_cut.loc[cnt.index, 'count'] = cnt.values

if spatial_coverage_cut.geometry.name != 'geometry':
    spatial_coverage_cut = spatial_coverage_cut.rename_geometry('geometry')


# 1. Simple Choropleth Map

In [ ]:
st_viz = st_visualizer(limit=len(spatial_coverage_cut))
st_viz.set_data(spatial_coverage_cut.dropna())

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")

st_viz.add_numerical_colormap('Viridis256', 'count', colorbar=True, cb_orientation='vertical', cb_location='right', label_standoff=12, border_line_color=None, location=(0,0))
st_viz.add_polygon(fill_color=st_viz.cmap, line_color=st_viz.cmap, fill_alpha=0.6, muted_alpha=0, legend_label=f'GPS Locations (Choropleth Map)')

st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bkhm.WheelZoomTool)

st_viz.show_figures(notebook=True, sizing_mode='stretch_both')

# 2. (Advanced) Integrating Filters to Choropleth Map

### Create a VISIONS instance for the Choropleth plot (main canvas)

In [ ]:
st_viz = st_visualizer(limit=len(spatial_coverage_cut))
st_viz.set_data(spatial_coverage_cut)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")

st_viz.add_numerical_colormap('Viridis256', 'count', colorbar=True, cb_orientation='vertical', cb_location='right', label_standoff=12, border_line_color=None, location=(0,0), nan_color=bokeh_colors.RGB(1,1,1,0))
st_viz.add_polygon(fill_color=st_viz.cmap, line_color=st_viz.cmap, fill_alpha=0.6, muted_alpha=0, legend_label=f'GPS Locations (Choropleth)')

### Create a VISIONS instance for keeping (& filtering) the original data

In [ ]:
data_points = st_visualizer(limit=len(gdf))
data_points.set_data(gdf)
data_points.set_figure(st_viz.figure)


categorical_name='vessel_id'

class Callback(viz_callbacks.BokehFilters):
    def __init__(self, vsn_instance, widget):
        super().__init__(vsn_instance, widget)
        
        
    def callback_prepare_data(self, new_pts, ready_for_output):
        self.vsn_instance.canvas_data = new_pts

        if ready_for_output:
            cnt = viz_helper.classify_area_proximity(self.vsn_instance.canvas_data, st_viz.data, compensate=True, verbose=True).area_id.value_counts()            
            st_viz.canvas_data = st_viz.data.loc[cnt.index].copy()
            st_viz.canvas_data.loc[:, 'count'] = cnt.values               
           
            st_viz.canvas_data = st_viz.prepare_data(st_viz.canvas_data)

            low, high = st_viz.canvas_data[st_viz.cmap['field']].agg(['min', 'max'])
            st_viz.cmap['transform'].low = 0 if low == high else low
            st_viz.cmap['transform'].high = high
                    
            st_viz.source.data = st_viz.canvas_data.drop(st_viz.canvas_data.geometry.name, axis=1).to_dict(orient="list")

            # print ('Releasing Lock...')
            st_viz.canvas_data = None
            self.vsn_instance.canvas_data = None
            self.vsn_instance.aquire_canvas_data = None
        
        
    def callback(self, attr, old, new):
        self.callback_filter_data()

        cat_value = self.widget.value
        new_pts = self.get_data()

        # print (cat_value, categorical_name)
        if cat_value:
            new_pts = new_pts.loc[new_pts[categorical_name] == cat_value].copy()

        self.callback_prepare_data(new_pts, self.widget.id==self.vsn_instance.aquire_canvas_data)

        
data_points.add_categorical_filter(title='Vehicle', categorical_name=categorical_name, height_policy='min', callback_class=Callback)

### Camera, Lights, Action

In [ ]:
data_points.figure.legend.location = "top_left"
data_points.figure.legend.click_policy = "mute"
data_points.figure.toolbar.active_scroll = data_points.figure.select_one(bkhm.WheelZoomTool)

In [ ]:
data_points.show_figures(notebook=True, sizing_mode='stretch_both')

# Plot Multiple Datasets at the Same Canvas

In [ ]:
df = pd.read_csv(r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv')
df.loc[:, 'datetime'] = pd.to_datetime(df.t, unit='ms')

df_algn_inter = pd.read_csv(r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_jun2017.csv')
df_algn_inter.loc[:, 'datetime'] = pd.to_datetime(df_algn_inter.t, unit='ms')

In [ ]:
st_viz = st_visualizer(limit=500)
st_viz.set_data(df, sp_columns=['lon', 'lat'])

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")


st_viz.add_marker(marker='circle', size=10, color='royalblue', alpha=0.5, fill_alpha=0.5, muted_alpha=0, legend_label=f'GPS Locations')
# st_viz.add_hover_tooltips(tooltips = [('Vessel_ID','@vessel_id'), ('Datetime','@datetime{%Y-%m-%d %H:%M:%S.%3N}'), ('Coordinates','(@lon, @lat)')], formatters={'@datetime': 'datetime'}, renderers=st_viz.renderers)

In [ ]:
st_viz2 = st_visualizer(limit=500)
st_viz2.set_data(df_algn_inter, sp_columns=['lon', 'lat'])

st_viz2.set_figure(st_viz.figure)
st_viz2.create_source()


st_viz2.add_marker(marker='square', size=10, color='orangered', alpha=0.5, fill_alpha=0.5, muted_alpha=0, legend_label=f'GPS Locations (interpolated - Kafka)')
st_viz2.add_hover_tooltips(tooltips = [('Vessel ID','@vessel_id'), ('Datetime','@datetime{%Y-%m-%d %H:%M:%S.%3N}'), ('Coordinates','(@lon, @lat)')], formatters={'@datetime': 'datetime'}, renderers=st_viz2.renderers)

In [ ]:
df_traj = viz_helper.create_linestring_from_points(viz_helper.create_geometry(df.copy()), ['vessel_id'])

st_viz3 = st_visualizer(limit=500)
st_viz3.set_data(df_traj, sp_columns=['lon', 'lat'])

st_viz3.set_figure(st_viz2.figure)
st_viz3.create_source()

st_viz3.add_line(line_type='multi_line', line_width=5, line_color='limegreen', alpha=0.5, muted_alpha=0, legend_label=f'Vessel Trajectories')

In [ ]:
st_viz3.figure.legend.location = "top_left"
st_viz3.figure.legend.click_policy = "mute"
st_viz3.figure.toolbar.active_scroll = st_viz3.figure.select_one(bkhm.WheelZoomTool)
st_viz3.show_figures(notebook=True, sizing_mode='stretch_both')

# Plot Multiple Datasets at Grid

In [ ]:
df = pd.read_csv(r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv')
df.loc[:, 'datetime'] = pd.to_datetime(df.t, unit='ms')

df_algn_inter = pd.read_csv(r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_jun2017.csv')
df_algn_inter.loc[:, 'datetime'] = pd.to_datetime(df_algn_inter.t, unit='ms')

In [ ]:
st_viz = st_visualizer(limit=1000)
st_viz.set_data(df)

tooltips = [('Vessel_id','@vessel_id'), ('Datetime','@t'), ('Coordinates','(@lon, @lat)')]
viz_express.plot_points_on_map(st_viz, tools=['lasso_select'], tooltips=tooltips)

In [ ]:
st_viz2 = st_visualizer(limit=1000)
st_viz2.set_data(df_algn_inter)

tooltips = [('Vessel ID','@vessel_id'), ('Datetime','@t'), ('Coordinates','(@lon, @lat)')]
viz_express.plot_points_on_map(st_viz2, tools=['lasso_select'], tooltips=tooltips, marker='triangle', color='red')

In [ ]:
st_viz2.figure.legend.location = "top_left"
st_viz2.figure.legend.click_policy = "mute"
st_viz2.figure.toolbar.active_scroll = st_viz2.figure.select_one(bkhm.WheelZoomTool)

In [ ]:
st_viz2.show_figures([[st_viz.figure, st_viz2.figure]], notebook=True, merge_tools=True, toolbar_location='right', sizing_mode='stretch_both')

# Visualize Data with Map alongside DataFrame

In [ ]:
st_viz = st_visualizer(limit=1000) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.get_data_csv(filepath=r'..\..\data\unipi_ais_dynamic_2017\unipi_ais_dynamic_dec2017.csv', nrows=10000)

st_viz.create_canvas(title=f'Prototype Plot', sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")

circ = st_viz.add_marker(marker='circle', size=10, color='royalblue', alpha=0.7, fill_alpha=0.5, muted_alpha=0, legend_label=f'Vessel GPS Locations')

tooltips = [('Vessel ID','@vessel_id'), ('Timestamp','@t'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon, @lat)')]

st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()
st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bkhm.WheelZoomTool)


columns = [
        bkhm.TableColumn(field="vessel_id", title="Vessel ID"),
        bkhm.TableColumn(field="speed", title="Speed"),
        bkhm.TableColumn(field="heading", title="Heading"),
        bkhm.TableColumn(field="lon", title="Longitude"),
        bkhm.TableColumn(field="lat", title="Latitude"),
        bkhm.TableColumn(field="t", title="Timestamp"),
        bkhm.TableColumn(field="course", title="COG")
        ]
data_table = bkhm.DataTable(source=st_viz.source, columns=columns, width=500, height=280)


st_viz.show_figures([[st_viz.figure, data_table]], width=1900, height=1900, sizing_mode='stretch_both', notebook=True)